In [1]:
%matplotlib inline

In [2]:
import torchvision

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys

lib_path = os.path.join('../src/lib')
if not lib_path in sys.path:
    sys.path.append(lib_path)

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data
from fhd_opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

torch.autograd.set_detect_anomaly(True)

Using tensorboardX


In [4]:
lib_path = '../../Rotated_IoU/'
if not lib_path in sys.path:
    sys.path.append(lib_path)
from utiles import box2corners
from oriented_iou_loss import cal_diou, cal_giou

In [7]:
args = ['eldet']
opt = opts().parse(args)
# opt.data_dir = '/data/cc/Data/CHD/detection/'
opt.data_dir = '/data/cc/Data/CHD/fake_data/'
opt.num_classes = 2
opt.dataset = 'coco_fhd'
opt.sincos_weight = 1

torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /data/cc/workspace/Repository/CenterNet/scripts/../../exp/eldet/default
heads {'hm': 2, 'reg': 2, 'l': 1, 'ratio_al': 1, 'ratio_bl': 1, 'theta': 1, 'sincos': 2}
Namespace(K=2, aggr_weight=0.0, agnostic_ex=False, angle_bins_weight=0, arch='dla_34', aug_ddd=0.5, aug_rot=0, batch_size=32, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[32], data_dir='/data/cc/Data/CHD/fake_data/', dataset='coco_fhd', debug=0, debug_dir='/data/cc/workspace/Repository/CenterNet/scripts/../../exp/eldet/default/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, ellipse_reg_weight=0, ellipse_weight=1, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/data/cc/workspace/Repository/CenterNet/scripts/../../exp/eldet', exp_id='default', fix_res=True,

In [8]:
logger = Logger(opt)

os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

In [9]:
print('Creating model...')
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(
      model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

Creating model...


In [10]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [11]:
print('Setting up data...')
val_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'val'), 
    batch_size=1, 
    shuffle=False,
    num_workers=1,
    pin_memory=True
)

if opt.test:
    _, preds = trainer.val(0, val_loader)
    val_loader.dataset.run_eval(preds, opt.save_dir)

train_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'train'), 
    batch_size=opt.batch_size, 
    shuffle=True,
    num_workers=opt.num_workers,
    pin_memory=True,
    drop_last=True
)


Setting up data...
==> initializing Fetal Heart Disease val data.
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loaded val 400 samples
==> initializing Fetal Heart Disease train data.
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Loaded train 1600 samples


In [12]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
logger = Logger(opt)
print('Starting training...')
best = 1e10
for epoch in range(start_epoch + 1, opt.num_epochs + 1):
    mark = epoch if opt.save_all else 'last'
    log_dict_train, _ = trainer.train(epoch, train_loader)
    logger.write('epoch: {} |'.format(epoch))
    print('epoch: {} |'.format(epoch))
    for k, v in log_dict_train.items():
        logger.scalar_summary('train_{}'.format(k), v, epoch)
        logger.write('{} {:8f} | '.format(k, v))
    if opt.val_intervals > 0 and epoch % opt.val_intervals == 0:
        save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(mark)), 
                 epoch, model, optimizer)
        with torch.no_grad():
            log_dict_val, preds = trainer.val(epoch, val_loader)
        for k, v in log_dict_val.items():
            logger.scalar_summary('val_{}'.format(k), v, epoch)
            logger.write('{} {:8f} | '.format(k, v))
        if log_dict_val[opt.metric] < best:
            best = log_dict_val[opt.metric]
            save_model(os.path.join(opt.save_dir, 'model_best.pth'), 
                   epoch, model)
    else:
        save_model(os.path.join(opt.save_dir, 'model_last.pth'), 
                 epoch, model, optimizer)
    logger.write('\n')
    if epoch in opt.lr_step:
        save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(epoch)), 
                 epoch, model, optimizer)
        lr = opt.lr * (0.1 ** (opt.lr_step.index(epoch) + 1))
        print('Drop LR to', lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
#     break
logger.close()

Starting training...


/home/cc/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/cc/miniconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/cc/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([32, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: CUDA error: device-side assert triggered

> /data/cc/workspace/Repository/CenterNet/src/lib/models/losses.py(181)forward()
    179         # loss = F.l1_loss(pred * mask, target * mask, reduction='elementwise_mean')
    180         loss = F.l1_loss(pred * mask, target * mask, size_average=False)
--> 181         loss = loss / (mask.sum() + 1e-4)
    182         return loss
    183 

ipdb> output
*** RuntimeError: cuda runtime error (710) : device-side assert triggered at /pytorch/aten/src/THC/THCCachingHostAllocator.cpp:278


In [ ]:
for iter_id, batch in enumerate(train_loader): # len(batch) = 6
    break

In [ ]:
# Designed for calculating Rotated IoU Loss
class IoULoss_test(nn.Module):
  def __init__(self):
    super(IoULoss_test, self).__init__()
  
  def forward(self, output, batch):
        
        
        
    bbox1 = torch.tensor([[[125, 125, 100, 50, math.pi]]]).cuda()
    bbox2 = torch.tensor([[[125, 125, 100, 50, 0.0 * math.pi]]]).cuda()
    print(bbox1.shape, bbox2.shape)
    print(bbox1, bbox2)
    iou_loss, iou = cal_diou(bbox1, bbox2)
    iou_loss, iou
     
    pred = _transpose_and_gather_feat(output, ind)
    mask = mask.unsqueeze(2).expand_as(pred).float()
    # loss = F.l1_loss(pred * mask, target * mask, reduction='elementwise_mean')
    loss = angle_loss(pred * mask, target * mask, size_average=False, use_smooth_l1=self.use_smooth_l1)
    loss = loss / (mask.sum() + 1e-4)
    return loss

In [ ]:
for batch in train_loader:
    batch_input = batch['input'].cuda()
    break

output = trainer.model_with_loss.model(batch_input)[-1]

In [ ]:
output, batch

In [ ]:
from models.decode import eldet_decode, _topk, _nms
def _gather_feat(feat, ind, mask=None):
    dim  = feat.size(2)
    ind  = ind.unsqueeze(2).expand(ind.size(0), ind.size(1), dim)
    feat = feat.gather(1, ind)
    if mask is not None:
        mask = mask.unsqueeze(2).expand_as(feat)
        feat = feat[mask]
        feat = feat.view(-1, dim)
    return feat

def _transpose_and_gather_feat(feat, ind):
    feat = feat.permute(0, 2, 3, 1).contiguous()
    feat = feat.view(feat.size(0), -1, feat.size(3))
    feat = _gather_feat(feat, ind)
    return feat

In [ ]:
def decode_output_4iou(output, K=2): # Decode output for iou loss
    heat = output['hm'].sigmoid_() # (B, 2, w, h)
    l = output['l'] # (B, 1, w, h)
    ratio_al = output["ratio_al"] # (B, 1, w, h)
    ratio_ba = output["ratio_ba"] # (B, 1, w, h)
    theta = output["theta"] # (B, 2, w, h)
    reg = output['reg'] if opt.reg_offset else None # (B, 2, w, h)

    batch, cat, height, width = heat.size()
    heat = _nms(heat)
    scores, inds, clses, ys, xs = _topk(heat, K=K)

    if reg is not None:
        reg = _transpose_and_gather_feat(reg, inds)  # (B, K, 1)
        reg = reg.view(batch, K, 2)
        xs = xs.view(batch, K, 1) + reg[:, :, 0:1]
        ys = ys.view(batch, K, 1) + reg[:, :, 1:2]
    else:
        xs = xs.view(batch, K, 1) + 0.5
        ys = ys.view(batch, K, 1) + 0.5

    l = _transpose_and_gather_feat(l, inds) # (B, K, 1)
    ratio_al = _transpose_and_gather_feat(ratio_al, inds) # (B, K, 1)
    ratio_ba = _transpose_and_gather_feat(ratio_ba, inds) # (B, K, 1)
    theta = _transpose_and_gather_feat(theta, inds) # (B, K, 1)

    a = l * ratio_al
    b = a * ratio_ba
    theta = theta * math.pi

    bboxes = torch.cat([xs, ys, 2*a, 2*b, theta], dim=2)
    return bboxes


In [ ]:
bbox_output = decode_output_4iou(output)
bbox_output.shape

In [ ]:
dets = eldet_decode(hm, l, ratio_al=ratio_al, ratio_ba=ratio_ba, theta=theta, 
                          reg=reg, cat_spec_wh=opt.cat_spec_wh, K=opt.K)

In [ ]:
xs = dets[..., 4:5]
ys = dets[..., 5:6]
l = dets[..., 6:7]
ratio_al = dets[..., 7:8]
ratio_ba = dets[..., 8:9]
xs.shape, ys.shape

In [ ]:
heat = _nms(batch['hm'])
reg = batch['reg']
l = batch['l']
ratio_al = batch['ratio_al']
ratio_ba = batch['ratio_ba']
theta = batch['theta']
scores, inds, clses, ys, xs = _topk(heat, K=2)

In [ ]:
import math
a = l * ratio_al
b = a * ratio_ba
theta = theta * math.pi

In [ ]:
def decode_label_4iou(label, K=2): # Decode label for iou loss
    heat = label['hm']
    reg = label['reg']
    l = label['l']
    ratio_al = label['ratio_al']
    ratio_ba = label['ratio_ba']
    theta = label['theta']
    batch, cat, height, width = heat.size()

    # perform nms on heatmaps
    heat = _nms(heat)
      
    scores, inds, clses, ys, xs = _topk(heat, K=K) # (B, K)
    if reg is not None:
        reg = reg.view(batch, K, 2)
        xs = xs.view(batch, K, 1) + reg[:, :, 0:1]
        ys = ys.view(batch, K, 1) + reg[:, :, 1:2]
    else:
        xs = xs.view(batch, K, 1) + 0.5
        ys = ys.view(batch, K, 1) + 0.5
    a = l * ratio_al
    b = a * ratio_ba
    theta = theta * math.pi

    bboxes = torch.cat([xs, ys, 2*a, 2*b, theta], dim=2)
    return bboxes

In [ ]:
bbox_label = decode_label_4iou(label).cuda()
bbox_output = decode_output_4iou(output).cuda()
print(bbox_label[0], bbox_output[0])
bbox_output.shape, bbox_output.requires_grad

In [ ]:
cal_diou(bbox_output, bbox_output)